In [1]:
import warnings

warnings.simplefilter("ignore")

from IPython.display import HTML, display

display(
    HTML(
        data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 95%; }
    div#maintoolbar-container { width: 95%; }
</style>
"""
    )
)


import json
import os

# Importing libraries
import pandas as pd
from ds.utilities.io import ds_trino

pd.options.display.max_columns = None
pd.options.display.max_rows = 50
pd.options.mode.chained_assignment = None


conn = ds_trino.create_trino_connection()

In [2]:
# VERY IMPORTAN: the query should include the catalag and the schema for every table e.g hudi_ingest.processed.fos_facility_view

query = """

with scan_out as ( SELECT
      CAST(order_uuid AS VARCHAR) AS order_id
      ,TRUE AS order_scanned_out
      ,count ((order_uuid)) as scanned_items
    FROM
     
      hudi_ingest.inventory_restaurant_platform_cockroach_dbreplica.inventory_transaction 
    WHERE
      day_partition>= DATE_FORMAT(DATE_TRUNC('month', current_date) - interval '18' month, '%Y-%m-%d')
      
      AND type = 'TRANSACTION_ORDER_SCANOUT'
      AND cause = 'CAUSE_MANUAL'
    GROUP BY
      1,2)



SELECT 
DATE(from_unixtime(reference_time/1000, facility_timezone)) AS created_date,
--que empiece en domingo
date_trunc('month', DATE(from_unixtime(reference_time/1000, facility_timezone))) as created_month,
date_trunc('week', DATE(from_unixtime((reference_time)/1000, facility_timezone))+ interval '1' day)- interval '1' day as created_week,
FLOOR((DATE_DIFF('day', DATE(created_at_ts), DATE(AT_TIMEZONE(current_timestamp, facility_timezone))) - 3) / 7) + 1 AS order_weeks_back_local,
cast(local_created_at_ts as time) as local_created_time,
facility_timezone,
'LATAM' as region,
dc.country,



CASE 

when dc.country='Brazil' then 'Brazil'

else 'SSL'

end as subregion,


(
CASE
   WHEN
      facility_id IN 
      (
         'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7',  -- LIM - Surquillo
         '6d2d117b-3fa9-4a2c-85b5-9b86a758f849'   -- LIM - Magdalena
      )
   THEN
      'Lima'
   WHEN
      facility_id IN 
      (
         'd65136fe-862a-4da5-81ef-580c28f8c61a'  -- UIO - 10 de Agosto
      )
   THEN
      'Quito'
   WHEN
      facility_id IN 
      (
         '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
         '199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
         '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
         'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
         'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
         '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
         'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
         '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82',  -- BOG - Cedritos
         '757182ff-7a54-4571-91d4-0027253c0117' --Salitre
      )
   THEN
      'Bogota'
  
  WHEN
      facility_id in 
      ('0016d9fc-4f27-4a05-88cb-90cd1cdf311f' --demoda
      )
  THEN 
      'Medellin'
      
      
      
   WHEN
      facility_id in 
      (
         '46fb5e19-3a79-4eda-b119-dee3a331fd81'  -- CLO - Chipichape
      )
   THEN
      'Cali' 
   WHEN
      facility_id in 
      (
         '58b818c6-615e-4316-b4a3-b9a6bb914c30'  --BAQ - Alto Prado
      )
   THEN
      'Barranquilla' 
   WHEN
      facility_id in 
      (
         '9eede066-b7e7-43c4-907c-21ece1688b14',  -- SJO - Guadalupe
         '967f0a2e-9c8e-4b35-929b-37420d9439c9'   -- SJO - Pavas
      )
   THEN
      'San Jose - CR' 
   WHEN
      facility_id in 
      (
         '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
         '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
         '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
         'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
         '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
         'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe
         'da27f06f-01fd-4106-88d9-b3fd41446515'  -- MEX - Lindavista
      )
   then
      'Mexico City'
   when
      facility_id in 
      (
         '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
         '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
         '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
         '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
         '88774bba-2e34-4771-8e64-8f059adf02a3'  -- MTY - Valle Oriente
      )
   then
      'Monterrey'
   when
      facility_id in 
      (
         'a9a6c4be-32b7-47cf-b783-7a5227a89ed4',  -- GDL - Ciudad del Sol
         'c66ce711-5486-433f-a9b7-1b36ae3a6dee' -- GDL - av mexico
      )
   then
      'Guadalajara'
   WHEN
      facility_id IN
      (
         '7b5c774f-7739-4d8e-a392-d49547d45fc3', -- BHZ - Contagem Industrial
         '554263c6-bde4-4d4c-92a7-21ed77e122cd', -- BHZ - Paracatu
         '522c745a-fa86-42de-83ca-227cbd3cb353', -- BHZ - Arthur Guimaraes
         '4518d99c-fa47-4da7-b5f5-c071c78b5ac2'  -- BHZ - Tancredo Neves'
      )
   then
      'Belo Horizonte' 
   WHEN
      facility_id IN
      (
         '81e6538d-c45b-4479-92e8-fc4e04373e12', -- RIO - Jose Bonifacio
         '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb', -- RIO - Barra
         '0ee5dccf-ed5b-465c-9858-b53351353e3b',  -- RIO - Camerino
         '976ae838-18e0-40bb-a8dc-360353c01b6d', -- RIO - Niteroi
         'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6' -- RIO - Tijuca
      )
   then
      'Rio de Janeiro' 
   WHEN
      facility_id IN
      (
         '6c601837-403c-425f-a51c-31428ed1a67e', -- SAO - Rua do Acre
         '87a40094-cf48-4032-8166-e4cee61bce2f', -- SAO - Dino
         '4326199f-628a-4b00-a6e6-054fb5182c64', -- SAO - Guararapes (Brooklin)
         'dcc461ae-c80c-46cc-a5dc-4a5a72761510', -- SAO - Lapa Clelia
         '7aae3115-311b-449f-88f4-0f1612ecfcee', -- SAO - Borba
         '1840180b-9a3b-4fb6-b35c-ef4e6785b26e', -- SAO - Abelardo
         '88aaaf99-a444-4e0f-99f6-267b635faa53',  -- SAO - Vargas
         'a63559d1-93c6-4f60-b683-1e207ccda10a', -- SAO - Batista
          '89cf86b9-9e74-4657-86df-2a851492af88' -- SAO - Guarulhos
      )
   then
      'Sao Paulo'
      
  WHEN 
        facility_id IN 
        (
        'c559b06b-e193-495c-85cd-a5d1608ca27d'  -- BSB - Aguas Claras
        )
    THEN
        'Brasilia'
      
      
    WHEN
        facility_id IN
        (
            'db3a588c-59ac-458e-84de-9e1c8ef5e3e8' -- 'SCL - Recoleta'
        )
    THEN
        'Santiago'
END
) AS city,
(
CASE
   WHEN facility_id = 'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7' THEN 'LIM - Surquillo'
   WHEN facility_id = '6d2d117b-3fa9-4a2c-85b5-9b86a758f849' THEN 'LIM - Magdalena'
   WHEN facility_id = 'd65136fe-862a-4da5-81ef-580c28f8c61a' THEN 'UIO - 10 de Agosto'
   WHEN facility_id = '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf' THEN 'BOG - Prado'
   WHEN facility_id = '4d78d1e1-5bce-4307-8175-51c816120fce' THEN 'BOG - Castellana'
   WHEN facility_id = 'b2f57126-8e64-4b11-964f-c497a3b53571' THEN 'BOG - Cocinas de la 73'
   WHEN facility_id = 'a5852445-e223-46c9-be99-ba33f204bcd1' THEN 'BOG - Puente Aranda'
   WHEN facility_id = '76a6ef30-ddb2-4178-8c37-86c0d10db4b3' THEN 'BOG - Atabanza'
   WHEN facility_id = 'dd16bf0e-8a89-4f21-bc02-1c9793227502' THEN 'BOG - San Felipe'
   WHEN facility_id = '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82' THEN 'BOG - Cedritos'
   when facility_id = '757182ff-7a54-4571-91d4-0027253c0117' Then 'BOG - Salitre 2 EM'
   when facility_id = '0016d9fc-4f27-4a05-88cb-90cd1cdf311f' THEN 'MED - DModa'
 
   
   WHEN facility_id = '46fb5e19-3a79-4eda-b119-dee3a331fd81' THEN 'CLO - Chipichape' 
   WHEN facility_id = '58b818c6-615e-4316-b4a3-b9a6bb914c30' THEN 'BAQ - Alto Prado' 
   WHEN facility_id = '9eede066-b7e7-43c4-907c-21ece1688b14' THEN 'SJO - Guadalupe' 
   WHEN facility_id = '967f0a2e-9c8e-4b35-929b-37420d9439c9' THEN 'SJO - Pavas'
   
   WHEN facility_id = '47e2d054-8b82-420c-b1b0-18010f57489c' THEN 'MEX - Insurgentes Sur'
   WHEN facility_id = '565273fe-0074-421d-b1e1-796d1e4390e1' THEN 'MEX - Napoles'
   WHEN facility_id = '864e0a3c-ecba-4305-b3ff-f22edd58c902' THEN 'MEX - Polanco'
   WHEN facility_id = 'bcb04949-3cda-4ca0-be39-b9f579e356ca' THEN 'MEX - Roma'
   WHEN facility_id = '9101d4f9-828f-45c5-b8b6-16986e76b374' THEN 'MEX - Pedregal'
   WHEN facility_id = 'b9545910-bca6-4b2a-a603-363721e7a23d' THEN 'MEX - Santa Fe'
   WHEN facility_id = 'da27f06f-01fd-4106-88d9-b3fd41446515' THEN 'MEX - Lindavista'
   
   WHEN facility_id = '394c0103-1c5f-424b-b398-68455fced5bb' THEN 'MTY - Cumbres'
   WHEN facility_id = '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f' THEN 'MTY - Leones NK'
   WHEN facility_id = '988c22e5-20c2-42a5-bc30-989c68ff5eee' THEN 'MTY - Tec'
   WHEN facility_id = '1b90b4ae-6d8f-4a73-bb9e-708c83387430' THEN 'MTY - Santa Maria'
   WHEN facility_id = '88774bba-2e34-4771-8e64-8f059adf02a3' THEN 'MTY - Valle Oriente'
   
   WHEN facility_id = 'a9a6c4be-32b7-47cf-b783-7a5227a89ed4' THEN 'GDL - Ciudad del Sol'
   WHEN facility_id = 'c66ce711-5486-433f-a9b7-1b36ae3a6dee' THEN 'GDL - Av. Mexico'
    
   WHEN facility_id = '7b5c774f-7739-4d8e-a392-d49547d45fc3' THEN 'BHZ - Contagem Industrial'
   WHEN facility_id = '554263c6-bde4-4d4c-92a7-21ed77e122cd' THEN 'BHZ - Paracatu'
   WHEN facility_id = '522c745a-fa86-42de-83ca-227cbd3cb353' THEN 'BHZ - Arthur Guimaraes' 
   WHEN facility_id = '4518d99c-fa47-4da7-b5f5-c071c78b5ac2' THEN 'BHZ - Tancredo Neves'
   
   WHEN facility_id = '81e6538d-c45b-4479-92e8-fc4e04373e12' THEN 'RIO - Jose Bonifacio'
   WHEN facility_id = '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb' THEN 'RIO - Barra' 
   WHEN facility_id = '0ee5dccf-ed5b-465c-9858-b53351353e3b' THEN 'RIO - Camerino'
   WHEN facility_id = '976ae838-18e0-40bb-a8dc-360353c01b6d' THEN 'RIO - Niteroi'
   WHEN facility_id = 'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6' THEN 'RIO - Tijuca'
   
   WHEN facility_id = '6c601837-403c-425f-a51c-31428ed1a67e' THEN 'SAO - Rua do Acre'
   WHEN facility_id = '87a40094-cf48-4032-8166-e4cee61bce2f' THEN 'SAO - Dino'
   WHEN facility_id = '4326199f-628a-4b00-a6e6-054fb5182c64' THEN 'SAO - Guararapes (Brooklin)'
   WHEN facility_id = 'dcc461ae-c80c-46cc-a5dc-4a5a72761510' THEN 'SAO - Lapa Clelia'
   WHEN facility_id = '7aae3115-311b-449f-88f4-0f1612ecfcee' THEN 'SAO - Borba'
   WHEN facility_id = '88aaaf99-a444-4e0f-99f6-267b635faa53' THEN 'SAO - Vargas'
   WHEN facility_id = '1840180b-9a3b-4fb6-b35c-ef4e6785b26e' THEN 'SAO - Abelardo'
   WHEN facility_id = 'a63559d1-93c6-4f60-b683-1e207ccda10a' THEN 'SAO - Batista'
   WHEN facility_id = '89cf86b9-9e74-4657-86df-2a851492af88' THEN 'SAO - Guarulhos'
  
   WHEN facility_id = 'c559b06b-e193-495c-85cd-a5d1608ca27d' THEN 'BSB - Aguas Claras'
   
   
  
 
   WHEN facility_id = 'db3a588c-59ac-458e-84de-9e1c8ef5e3e8' THEN 'SCL - Recoleta'

  
END
) AS facility_sf_name,
facility_name,
organization_name as org_name,
organization_id,
co.order_id,
facility_id,
order_display_id,
ofo_slug, store_link_service_slug,
facility_country_code as country_code, 
is_cancelled,
oc_order_issue_count,
order_issue_details,
hoi_order_issue_details,
oc_order_issue_details,
order_issue_types,
hoi_order_issue_types,
oc_order_issue_types,

subtotal,
total,

commission, discount	tip, tip_for_restaurant, tip_for_courier, delivery_fee, marketing_fee, service_fee

,ROW_NUMBER() OVER (PARTITION BY ofo_slug, external_order_id, order_items_quantity, subtotal ORDER BY co.order_id) as row_number

,cast(local_accepted_at_last_ts as TIME ) as accepted_local_time,
local_processed_at_ts,local_ordered_at_ts,local_accepted_at_ts, local_in_handoff_at_ts,
local_handed_off_at_ts,local_printed_at_ts,local_trashed_at_ts,local_pickup_picked_up_at_ts,
local_courier_checked_in_at_ts,local_courier_dropped_off_at_ts,external_order_id,fulfillment_state,fulfillment_mode,
pickup_state,scheduling_type,ofo_status,order_status
,cancellation_state,cancellation_reason as cancel_reason_super_set
,ofo_cancellation_reason,order_rating, order_review_full_text,location_state,

case when location_state='LOCATION_HANDED_OFF' then 1

else 0

end

as is_handed_off_num,

CASE
        WHEN is_cancelled THEN 'cancel'
        WHEN order_issue_types IS NOT NULL THEN 'issue'
        ELSE 'success'
        END AS defect_type,

CASE
        WHEN is_cancelled AND cancellation_reason = 'UNAVAILABLE_ITEM' THEN 'item_unavailable'
        WHEN is_cancelled AND confirmation_state <> 'CONFIRMATION_CONFIRMED' THEN 'confirmation_failed'
        WHEN is_cancelled AND  location_state!='LOCATION_HANDED_OFF' THEN 'handoff_failed'
        WHEN is_cancelled AND time_to_last_in_handoff_millis > 1200000 THEN 'pick_over_20' --Pick time was over 20 minutes long
        WHEN is_cancelled THEN 'other_cancel'
        WHEN array_join(order_issue_types, ', ') LIKE '%WRONG_ORDER%' THEN 'order_wrong'
        WHEN array_join(order_issue_types, ', ') LIKE '%MISSING_ITEM%' OR array_join(order_issue_types, ', ') LIKE '%WRONG_QUANTITY%' THEN 'item_missing'
        WHEN array_join(order_issue_types, ', ') LIKE '%WRONG_ITEM%' OR array_join(order_issue_types, ', ') LIKE '%WRONG_SIZE%' THEN 'item_wrong'
        WHEN array_join(order_issue_types, ', ') LIKE '%INCOMPLETE%' THEN 'order_incomplete'
        ELSE 'other_issue'
      END AS defect_reason,

COALESCE(IF(adjustment IS NOT NULL AND (is_cancelled OR order_issue_types IS NOT NULL), adjustment < 0), FALSE) AS is_refunds
,time_to_last_in_handoff_millis BETWEEN 1 AND 1200000 AS order_20picked --TRUE if picked in under 20 minutes

,order_source, hoi_only_order_type,order_items_quantity,scan_out.order_scanned_out,


printed_at_ts, in_handoff_at_ts, handed_off_at_ts, courier_checked_in_at_ts, courier_dropped_off_at_ts, courier_arrived_at_ts

,scanned_items



FROM hudi_ingest.analytics_views.customer_orders_with_hoi_curr co
JOIN hudi_ingest.scratch.dim_countries dc 
   ON co.facility_country_code = dc.country_code AND region = 'LATAM'
   
left join scan_out
on co.order_id=scan_out.order_id






--aqui se filtra la fecha    

WHERE DATE(from_unixtime(reference_time/1000, facility_timezone))>= current_date

--and co.day_partition >= DATE_FORMAT(current_date, '%Y-%m-%d')
and co.day_partition >='01-01-2022'
AND reference_time/1000 IS NOT NULL 
AND DATE(from_unixtime(reference_time/1000, facility_timezone)) >= DATE '2021-01-15'
AND tenant_type in( 'VIRTUAL','CLOUD_RETAIL')
--AND organization_name='Tere Cazola'
--AND is_cancelled

AND facility_id IN ( -- 
    -- COLOMBIA
        -- BOGOTA
            '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
          --'199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
            '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
            'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
            'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
            '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
            'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
            '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82', -- BOG - Cedritos
            '757182ff-7a54-4571-91d4-0027253c0117', --BOG salitre
            --medellin
            '0016d9fc-4f27-4a05-88cb-90cd1cdf311f',--de moda
        -- CALI    
            '46fb5e19-3a79-4eda-b119-dee3a331fd81', -- CLO - Chipichape
        -- BARRANQUILLA
            '58b818c6-615e-4316-b4a3-b9a6bb914c30', -- BAQ - Alto Prado
    -- COSTA RICA




-- MEXICO
        -- MEXICO CITY
            '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
            '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
            '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
            'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
            '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
            'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe
            'da27f06f-01fd-4106-88d9-b3fd41446515', -- MEX - Lindavista
        -- MONTERREY
            '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
            '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
            '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
            '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
            '88774bba-2e34-4771-8e64-8f059adf02a3', -- MTY - Valle Oriente
        -- GUADALAJARA
            'a9a6c4be-32b7-47cf-b783-7a5227a89ed4', -- GDL - Ciudad del Sol
            'c66ce711-5486-433f-a9b7-1b36ae3a6dee' -- GDL - av mexico
            

                
                )


"""

print("querying ...")
super_set = ds_trino.fetch_data(conn=conn, sql_str=query, use_cache=False)
print("pulled")

querying ...
pulled


In [3]:
super_set

,created_date,created_month,created_week,order_weeks_back_local,local_created_time,facility_timezone,region,country,subregion,city,facility_sf_name,facility_name,org_name,organization_id,order_id,facility_id,order_display_id,ofo_slug,store_link_service_slug,country_code,is_cancelled,oc_order_issue_count,order_issue_details,hoi_order_issue_details,oc_order_issue_details,order_issue_types,hoi_order_issue_types,oc_order_issue_types,subtotal,total,commission,tip,tip_for_restaurant,tip_for_courier,delivery_fee,marketing_fee,service_fee,row_number,accepted_local_time,local_processed_at_ts,local_ordered_at_ts,local_accepted_at_ts,local_in_handoff_at_ts,local_handed_off_at_ts,local_printed_at_ts,local_trashed_at_ts,local_pickup_picked_up_at_ts,local_courier_checked_in_at_ts,local_courier_dropped_off_at_ts,external_order_id,fulfillment_state,fulfillment_mode,pickup_state,scheduling_type,ofo_status,order_status,cancellation_state,cancel_reason_super_set,ofo_cancellation_reason,order_rating,order_review_full_text,location_state,is_handed_off_num,defect_type,defect_reason,is_refunds,order_20picked,order_source,hoi_only_order_type,order_items_quantity,order_scanned_out,printed_at_ts,in_handoff_at_ts,handed_off_at_ts,courier_checked_in_at_ts,courier_dropped_off_at_ts,courier_arrived_at_ts,scanned_items
0,2023-09-13,2023-09-01,2023-09-10,1,13:56:55.000,America/Bogota,LATAM,Colombia,SSL,Medellin,MED - DModa,DeModa Poblado,Franui,d44c0ca0-acf4-47b3-b21f-b5106935ceeb,7fe91352-4d85-3091-b8be-fe2b32795490,0016d9fc-4f27-4a05-88cb-90cd1cdf311f,xfkjy,rappi,rappi,CO,False,NaN,None,None,None,None,None,None,55800.0,70800.0,None,NaN,NaN,4500.0,10500.0,None,None,1,13:56:56.000,2023-09-13 14:27:17.000 America/Bogota,2023-09-13 13:48:37.000 America/Bogota,2023-09-13 13:56:56.000 America/Bogota,None,2023-09-13 14:04:10.000 America/Bogota,None,None,2023-09-13 14:04:11.000 America/Bogota,None,2023-09-13 14:04:31.000 America/Bogota,2163545496,FULFILLMENT_FULFILLED,FULFILLMENT_MODE_OFO_DELIVERY,PICKUP_PICKED_UP,SCHEDULING_ASAP,OFO_STATUS_FULFILLED,OFO_STATUS_FULFILLED,None,None,None,None,None,LOCATION_HANDED_OFF,1,success,other_issue,False,None,rappi,None,1,True,None,None,2023-09-13 19:04:10.000,None,2023-09-13 19:04:31.000,None,1.0
1,2023-09-13,2023-09-01,2023-09-10,1,13:26:34.000,America/Bogota,LATAM,Colombia,SSL,Bogota,BOG - Castellana,SA-CO-Bogotá-Castellana,Franui,d44c0ca0-acf4-47b3-b21f-b5106935ceeb,9ecdde35-9520-39c9-99bc-71d1b0b90558,4d78d1e1-5bce-4307-8175-51c816120fce,zrnio,rappi,rappi,CO,False,NaN,None,None,None,None,None,None,27900.0,32800.0,None,NaN,NaN,0.0,4900.0,None,None,1,13:26:34.000,2023-09-13 14:03:14.000 America/Bogota,2023-09-13 13:12:27.000 America/Bogota,2023-09-13 13:26:34.000 America/Bogota,2023-09-13 13:27:03.000 America/Bogota,2023-09-13 13:57:25.000 America/Bogota,None,None,2023-09-13 13:57:26.000 America/Bogota,None,2023-09-13 13:57:57.000 America/Bogota,2163535268,FULFILLMENT_FULFILLED,FULFILLMENT_MODE_OFO_DELIVERY,PICKUP_PICKED_UP,SCHEDULING_ASAP,OFO_STATUS_FULFILLED,OFO_STATUS_FULFILLED,None,None,None,None,None,LOCATION_HANDED_OFF,1,success,other_issue,False,True,rappi,None,1,True,None,2023-09-13 18:27:03.000,2023-09-13 18:57:25.000,None,2023-09-13 18:57:57.000,2023-09-13 18:27:41.000,1.0
2,2023-09-13,2023-09-01,2023-09-10,1,08:03:48.000,America/Mexico_City,LATAM,Mexico,SSL,Mexico City,MEX - Lindavista,NA-MX-CDMX-Lindavista,Tere Cazola,b556b267-8821-4162-a0cc-254fc70dafa0,c1bbf1ea-bbe3-3dbf-bec2-ee6627def6a1,da27f06f-01fd-4106-88d9-b3fd41446515,555kn,ubereats,ubereats-api,MX,False,NaN,None,None,None,None,None,None,740.0,740.0,None,NaN,NaN,NaN,NaN,None,None,1,08:03:50.000,2023-09-13 10:18:54.000 America/Mexico_City,2023-09-13 08:03:47.000 America/Mexico_City,2023-09-13 08:03:50.000 America/Mexico_City,2023-09-13 08:12:23.000 America/Mexico_City,2023-09-13 08:17:53.000 America/Mexico_City,None,None,2023-09-13 08:17:53.000 America/Mexico_City,2023-09-13 08:08:54.000 America/Mexico_City,None,244e14e4-c0c6-43b6-bff1-5bbcee6587d3,FULF

In [4]:
super_set.columns

Index(['created_date', 'created_month', 'created_week',
       'order_weeks_back_local', 'local_created_time', 'facility_timezone',
       'region', 'country', 'subregion', 'city', 'facility_sf_name',
       'facility_name', 'org_name', 'organization_id', 'order_id',
       'facility_id', 'order_display_id', 'ofo_slug',
       'store_link_service_slug', 'country_code', 'is_cancelled',
       'oc_order_issue_count', 'order_issue_details',
       'hoi_order_issue_details', 'oc_order_issue_details',
       'order_issue_types', 'hoi_order_issue_types', 'oc_order_issue_types',
       'subtotal', 'total', 'commission', 'tip', 'tip_for_restaurant',
       'tip_for_courier', 'delivery_fee', 'marketing_fee', 'service_fee',
       'row_number', 'accepted_local_time', 'local_processed_at_ts',
       'local_ordered_at_ts', 'local_accepted_at_ts', 'local_in_handoff_at_ts',
       'local_handed_off_at_ts', 'local_printed_at_ts', 'local_trashed_at_ts',
       'local_pickup_picked_up_at_ts', 'local_co

In [5]:
from google.oauth2 import service_account
from ds.utilities.secrets import secrets
import pygsheets
SCOPES = ('https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive')
s = secrets.Secrets()
credentials_sheets = service_account.Credentials.from_service_account_info(s.get_secret("cloud_retail_latam_service_account"),scopes=SCOPES)
#gc = 

In [6]:
#import pygsheets
## GSheets Oauth
print('Init Auth')
gc = pygsheets.authorize(custom_credentials=credentials_sheets )


# open the spreedsheet
print('Opening Spreadsheet')

ss = gc.open_by_key('1ynp6YE3mrboKISzH0aE3-gTd8d_vE8jD47qtYcn-1WU')

# select sheet

ws = ss.worksheet('title','today_orders')

print('Clearing Range ...')
ws.clear(start='A', end='cc')
print('Range CLEARED')

print('Pasting Info ...')
ws.set_dataframe(super_set, start="A1",copy_head=True, extend=True, nan= '' )
print('UPDATED!!')

Init Auth
Opening Spreadsheet
Clearing Range ...
Range CLEARED
Pasting Info ...
UPDATED!!


In [7]:
super_set['defect_reason'].value_counts()

other_issue       136
handoff_failed      2
Name: defect_reason, dtype: int64

In [9]:
from ds.utilities.secrets import secrets
from google.oauth2 import service_account
from google.cloud import bigquery
s = secrets.Secrets()
credentials = service_account.Credentials.from_service_account_info(s.get_secret("cloud_retail_latam_service_account"))
client = bigquery.Client(credentials=credentials)

In [10]:
big_query="""
select 
date(created_date) as created_date, link, country,facility_sf_name, facility_id, org_name, subtotal, subtotal_usd, order_id, ofo_slug, order_issue_details, ofo_cancellation_reason, facility_cancellation_reason,
Aggregated_root_cause, new_defect_reason,defect_type, defect_inv


  FROM `css-operations.latam_cr_growth.fulfillment` 


where date(created_date) >=(current_date- interval '33' day)


and Aggregated_root_cause='Confirmation failed'

and country in ('Mexico','Colombia')

"""

In [11]:
failed= pd.read_gbq(query=big_query, credentials=credentials,dialect='standard')

In [12]:
#failed=super_set[super_set['defect_reason']=='confirmation_failed']#

In [13]:
failed

,created_date,link,country,facility_sf_name,facility_id,org_name,subtotal,subtotal_usd,order_id,ofo_slug,order_issue_details,ofo_cancellation_reason,facility_cancellation_reason,Aggregated_root_cause,new_defect_reason,defect_type,defect_inv
0,2023-09-07,https://tools.cloudkitchens.com/order-inspecto...,Mexico,MEX - Roma,bcb04949-3cda-4ca0-be39-b9f579e356ca,Cookievery,230.0,13.434579,3e8f11c3-ef5a-3fd5-a33e-346c41ea1639,total-play,None,None,None,Confirmation failed,confirmation_failed,cancel,Sku storelink broken
1,2023-08-26,https://tools.cloudkitchens.com/order-inspecto...,Colombia,BOG - Castellana,4d78d1e1-5bce-4307-8175-51c816120fce,Nicookies,35000.0,8.602616,072d0872-849f-311e-b9c1-7d96f7af2653,rappi,None,None,None,Confirmation failed,confirmation_failed,cancel,other
2,2023-08-24,https://tools.cloudkitchens.com/order-inspecto...,Colombia,MED - DModa,0016d9fc-4f27-4a05-88cb-90cd1cdf311f,Franui,27900.0,6.857514,9da5aeeb-5ae0-3f7a-9454-92379b6bd67e,rappi,None,None,None,Confirmation failed,confirmation_failed,cancel,other
3,2023-08-19,https://tools.cloudkitchens.com/order-inspecto...,Colombia,BOG - Salitre 2 EM,757182ff-7a54-4571-91d4-0027253c0117,Las Negras Reposteria,25800.0,6.341357,a5c73a79-5fc6-38cc-ad3e-8df65ecc99d3,rappi,None,None,None,Confirmation failed,confirmation_failed,cancel,other
4,2023-08-22,https://tools.cloudkitchens.com/order-inspecto...,Mexico,MEX - Pedregal,9101d4f9-828f-45c5-b8b6-16986e76b374,La Michoacana Delivery,258.0,15.194346,cb73308b-d4c3-3034-9c70-026488375875,didifood,None,RESTAURANT_TOO_BUSY,None,Confirmation failed,confirmation_failed,cancel,other
5,2023-08-12,https://tools.cloudkitchens.com/order-inspecto...,Mexico,MEX - Pedregal,9101d4f9-828f-45c5-b8b6-16986e76b374,La Michoacana Delivery,98.0,5.771496,f5a85cfb-e9a0-3e24-b364-41de3ca4f721,didifood,None,RESTAURANT_TOO_BUSY,None,Confirmation failed,confirmation_failed,cancel,other
6,2023-08-31,https://tools.cloudkitchens.com/order-inspecto...,Mexico,MEX - Pedregal,9101d4f9-828f-45c5-b8b6-16986e76b374,La Michoacana Delivery,129.0,7.597173,6a51d051-7950-337c-b455-07342498e546,didifood,None,None,None,Confirmation failed,confirmation_failed,cancel,other
7,2023-08-19,https://tools.cloudkitchens.com/order-inspecto...,Mexico,MEX - Lindavista,da27f06f-01fd-4106-88d9-b3fd41446515,Tere Cazola,360.0,21.201413,132a3146-0ba4-3848-bfe3-c576d65261c7,ubereats,None,None,REASON_OTHER,Confirmation failed,confirmation_failed,cancel,Inventory variance
8,2023-09-12,https://tools.cloudkitchens.com/order-inspecto...,Mexico,MEX - Santa Fe,b9545910-bca6-4b2a-a603-363721e7a23d,Flor de Lis CR,225.0,13.142523,7fa07d34-f5ae-34cc-af82-b3e5a5ceff1d,ubereats,None,UNACCEPTED_ORDER,None,Confirmation failed,confirmation_failed,cancel,other
9,2023-09-03,https://tools.cloudkitchens.com/order-inspecto...,Mexico,MEX - Pedregal,9101d4f9-828f-45c5-b8b6-16986e76b374,Flor de Lis CR,225.0,13.142523,d03f2582-0b20-3fbc-9cf3-822eb6d64907,ubereats,None,UNACCEPTED_ORDER,None,Confirmation failed,confirmation_failed,cancel,Sku storelink broken


In [14]:
#import pygsheets
## GSheets Oauth
print('Init Auth')
gc = pygsheets.authorize(custom_credentials=credentials_sheets )


# open the spreedsheet
print('Opening Spreadsheet')

ss = gc.open_by_key('1ynp6YE3mrboKISzH0aE3-gTd8d_vE8jD47qtYcn-1WU')

# select sheet

ws = ss.worksheet('title','Import failed')

print('Clearing Range ...')
ws.clear(start='A', end='cc')
print('Range CLEARED')

print('Pasting Info ...')
ws.set_dataframe(failed, start="A1",copy_head=True, extend=True, nan= '' )
print('UPDATED!!')

Init Auth
Opening Spreadsheet
Clearing Range ...
Range CLEARED
Pasting Info ...
UPDATED!!
